In [2]:
import os
import re
import sklearn
import numpy as np
import pandas as pd
from wombat.engine import ml_model
from wombat.engine import parse_input_description as parse_title
from wombat.models import Item
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestRegressor
from wombat.models import dbsession, engine, ItemType, ItemAdjective
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (20,12)

In [3]:
model_dir = '/home/patrick/Dropbox/insight/wombat/wombat/engine/stat_model_pickles'
model = os.path.join(model_dir, 'rfr_v0.3_with_adj.pkl')
clf = joblib.load(model)

In [4]:
df = ml_model.canonical_df

# split data into train and test group
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75

# copy of train before we delete the 'rent_per_week' and 'title' field. We want title 
# we will need these things before we make dummy variables
df_full_training = df[df['is_train']==False]

# split data into train and test group
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
features = df.columns.drop(['rent_per_week', 'title']) # just column names

# get one-hot columns for brands 
dummy_brands = pd.get_dummies(df['brand'])
df = pd.concat([df, dummy_brands], axis = 1)

df = df.drop('brand', axis = 1)

# get one-hot columns for item_types
dummified_items = pd.get_dummies(df['item_type'])
df = pd.concat([df, dummified_items], axis = 1)
df = df.drop('item_type', axis = 1)
df_brands = df

# get list of adjective features form database
features_adj = [f.name for f in dbsession.query(ItemAdjective).all()]
item_types   = [item.name for item in dbsession.query(ItemType).all()]

tokenized_titles = []
for title in df['title']:
    parse_title.create_one_hot_row_adj(title, features_adj)
    tokenized_titles.append(parse_title.create_one_hot_row_adj(title, features_adj))
df_adj = pd.DataFrame(tokenized_titles, columns = features_adj)

features_adj = [f.name for f in dbsession.query(ItemAdjective).all()]
item_types   = [item.name for item in dbsession.query(ItemType).all()]

tokenized_titles = []
for title in df['title']:
    parse_title.create_one_hot_row_adj(title, features_adj)
    tokenized_titles.append(parse_title.create_one_hot_row_adj(title, features_adj))
df_adj = pd.DataFrame(tokenized_titles, columns = features_adj)

df = pd.concat([df_brands, df_adj], axis = 1)
features = df.columns.drop(['rent_per_week', 'title']) # just column names

In [5]:
df
train, test = df[df['is_train']==True], df[df['is_train']==False]

In [6]:
test[features]

,cost,is_train,Dolce & Gabbana,Adrianna Papell,Alexia Admor,Amanda Uprichard,Basix II,Clover Canyon,Narciso Rodriguez,Trina Turk,...,festival,paradise,boston,label,soon,girl,one,off,top,jumper
1,695.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,349.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,400.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,229.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,225.00,False,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
20,304.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,1300.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,282.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,400.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31,480.00,False,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df = ml_model.canonical_df
df.iloc[1]
brand = 'Tibi'
item_type = "dresses"
title = 'Lurex Dress'

In [8]:
from wombat.engine import ml_model
from wombat.engine.parse_input_description import create_one_hot_row_adj
from wombat.engine.parse_input_description import create_one_hot_row

In [13]:
brands_one_hot = create_one_hot_row(brand, ml_model.brands)

item_types_query = "SELECT * FROM item_types;"
item_types = [i[1] for i in engine.execute(item_types_query).fetchall()]
item_types_one_hot = create_one_hot_row(item_type, item_types)

adjectives_query = "SELECT * FROM item_adjectives;"
adjectives = [a[1] for a in engine.execute(adjectives_query).fetchall()]
adjectives_one_hot = create_one_hot_row_adj(title, adjectives)

columns = ml_model.brands + item_types + adjectives
print(len(ml_model.brands))
print(len(item_types))
print(len(adjectives))
print("{}+{}+{} = {}".format(len(ml_model.brands), len(item_types), len(adjectives), len(ml_model.brands+item_types+adjectives)))
one_hot_row = brands_one_hot + item_types_one_hot + adjectives_one_hot
print(ml_model.brand_length)
#columns = adjectives
#one_hot_row = adjectives_one_hot
#one_hot_row

1698
11
112
1698+11+112 = 1821
1698


In [14]:
input_df = pd.DataFrame(one_hot_row, columns)
rfr_predicted = clf.predict(input_df.T).astype(int)
print(rfr_predicted)
#input_df.T

[36]
